In [2]:
#Examining the index object
import pandas as pd
college=pd.read_csv('data/college.csv')
column=college.columns
column
column.values
column[-1]
column[[0,2]]
column+'_A'

c1=column[:4]
c2=column[2:6]
c1|c2
c1^c2

Index(['CITY', 'INSTNM', 'MENONLY', 'WOMENONLY'], dtype='object')

In [25]:
#Producing Certesian products
import numpy as np
s1=pd.Series(index=list('aaad'),data=np.arange(4))
s1
s2=pd.Series(index=list('cababb'),data=np.arange(6))
s2  #s1的每个a以此去paris  s2的a
s1+s2

a    1.0
a    3.0
a    2.0
a    4.0
a    3.0
a    5.0
b    NaN
b    NaN
b    NaN
c    NaN
d    NaN
dtype: float64

In [3]:
#Exploding indexes
employee=pd.read_csv('data/employee.csv',index_col='RACE')
salary1=employee['BASE_SALARY'].copy()
salary2=employee['BASE_SALARY'].copy()
salary1 is salary2 #check whether those two are the same, there are not same, becase without copy, if you chage one, another one will change t
#too, so we use copy method to create brand new copy of the data
salary1=salary1.sort_index()
salary_add=salary1+salary2 #a Cartesian product took place
len(salary1),len(salary2),len(salary_add) 

index_vc=salary1.index.value_counts(dropna=False)
index_vc

Black or African American            700
White                                665
Hispanic/Latino                      480
Asian/Pacific Islander               107
NaN                                   35
American Indian or Alaskan Native     11
Others                                 2
Name: RACE, dtype: int64

In [45]:
#Filling values with unequal indexes
baseball_14=pd.read_csv('data/baseball14.csv',index_col='playerID')
baseball_15=pd.read_csv('data/baseball15.csv',index_col='playerID')
baseball_16=pd.read_csv('data/baseball16.csv',index_col='playerID')
baseball_14.head()

baseball_14.index.difference(baseball_15.index) #check which playerID in 14 but not in 15

hits_14=baseball_14['H']
hits_15=baseball_15['H']
hits_16=baseball_16['H']

#hits_14+hits_15 #如果你用这用的话，那么那些在没有在14里确实15里的player的值会变成NaN Cartesian product 

hits_14.add(hits_15,fill_value=0).head() #fill_value to avoid missing values
hit_total=hits_14.add(hits_15,fill_value=0).add(hits_16,fill_value=0)
hit_total.hasnans


#Add DataFrame
df_14=baseball_14[['G','AB','R','H']]
df_15=baseball_15[['G','AB','R','H']]
#(df_14+df_15).style.highlight_null('yellow') 
df_14.add(df_15,fill_value=0).style.highlight_null('yellow')

,G,AB,R,H
playerID,,,,
altuvjo01,312,1298,171,425
cartech02,274,898,118,193
castrja01,230,802,81,174
congeha01,73,201,25,46
corpoca01,55,170,22,40
correca01,99,387,52,108
dominma01,157,564,51,121
fowlede01,116,434,61,120
gattiev01,153,566,66,139


In [9]:
#Appedning columns from diffrent DataFrame
import pandas as pd
employee=pd.read_csv('data/employee.csv')
dept_sal=employee[['DEPARTMENT','BASE_SALARY']]
dept_sal=dept_sal.sort_values(['DEPARTMENT','BASE_SALARY'],ascending=[True,False])
max_dept_sal=dept_sal.drop_duplicates(subset='DEPARTMENT') #keep the first row of each department
max_dept_sal=max_dept_sal.set_index('DEPARTMENT')
employee=employee.set_index('DEPARTMENT')
#append a column
employee['MAX_DEPT_SALARY']=max_dept_sal['BASE_SALARY']
employee.head()
employee.query('BASE_SALARY>MAX_DEPT_SALARY')

,UNIQUE_ID,POSITION_TITLE,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE,MAX_DEPT_SALARY
DEPARTMENT,,,,,,,,,,


In [1]:
#Highling the maximun value from each column
import numpy as np
import pandas as pd
college=pd.read_csv('data/college.csv',index_col='INSTNM')
college.dtypes
college.MD_EARN_WNE_P10.iloc[0] #string
cols=['MD_EARN_WNE_P10','GRAD_DEBT_MDN_SUPP']
for col in cols:
    college[col]=pd.to_numeric(college[col],errors='coerce') #make 不能转化的成NaN

college_n=college.select_dtypes(include=[np.number])
college_n

#some columns only have binary (0/1) valuesm, we should take them out
condition=college_n.nunique()==2
binary_cols=college_n.columns[condition].tolist()
binary_cols

college_n2=college_n.drop(labels=binary_cols,axis=1) #axis='columns'

max_cols=college_n2.idxmax() #use idxmax to find the index label of the maximun value for each column
unique_max_cols=max_cols.unique()
unique_max_cols
college_n2.loc[unique_max_cols].style.highlight_max()


#by default the highlight_max method highlights the max value of each column, we can change the axis parameter to highlight the max value of each 
#row instead
college_ugds=college.filter(like='UGDS_').head(10)
college_ugds.style.highlight_max(axis='columns')


,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.26,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.01
Amridge University,0.299,0.4192,0.0069,0.0034,0,0,0,0,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.035
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.001,0.0006,0.0098,0.0243,0.0137
The University of Alabama,0.7825,0.1119,0.0348,0.0106,0.0038,0.0009,0.0261,0.0268,0.0026
Central Alabama Community College,0.7255,0.2613,0.0044,0.0025,0.0044,0,0,0,0.0019
Athens State University,0.7823,0.12,0.0191,0.0053,0.0157,0.001,0.0174,0.0057,0.0334
Auburn University at Montgomery,0.5328,0.3376,0.0074,0.0221,0.0044,0.0016,0.0297,0.0397,0.0246


In [20]:
#Replicating idxman with method chaining
import pandas as pd
import numpy as np
college=pd.read_csv('data/college.csv',index_col='INSTNM')
cols=['MD_EARN_WNE_P10','GRAD_DEBT_MDN_SUPP']
for col in cols:
    college[col]=pd.to_numeric(college[col],errors='coerce')
    
college_n=college.select_dtypes(include=[np.number])
college_n

#some columns only have binary (0/1) valuesm, we should take them out
condition=college_n.nunique()==2
binary_cols=college_n.columns[condition].tolist()
binary_cols

college_n=college_n.drop(labels=binary_cols,axis=1) 

college_n.max()

college_n.eq(college_n.max()).head() #use eq to test each values with its columns max.
has_at_least_on_true=college_n.eq(college_n.max()).any(axis=1)

college_n.eq(college_n.max()).cumsum() #可见比如UGDS_WHITE有109个符合的

has_row_max2=college_n.eq(college_n.max()).cumsum().cumsum().eq(1).any(axis=1)
has_row_max2.sum()
idxmax_col=has_row_max2[has_row_max2].index
idxmax_col

set(college_n.idxmax().unique())==set(idxmax_col)    #idxmax() return the first orrence of the max values rows

True

In [31]:
#Finding the most common maximun
college_ugds=college.filter(like='UGDS_')
highest_percent_race=college_ugds.idxmax(axis=1) #use the idxmax method to get the higest values for each row
highest_percent_race.value_counts(normalize=True)

#在黑鬼最多的学校，第二多种族的比列
college_black=college_ugds[highest_percent_race=='UGDS_BLACK'] 
college_black=college_black.drop('UGDS_BLACK',axis=1)
college_black.idxmax(axis=1).value_counts(normalize=True)

UGDS_WHITE    0.661228
UGDS_HISP     0.230326
UGDS_UNKN     0.071977
UGDS_NRA      0.018234
UGDS_ASIAN    0.009597
UGDS_2MOR     0.006718
UGDS_NHPI     0.000960
UGDS_AIAN     0.000960
dtype: float64